In [1]:

#help(Client)

In [2]:
import os
import os
import sys
import pprint
import logging

from rucio.client import Client
from rucio.common.exception import (DuplicateRule, DataIdentifierAlreadyExists, DuplicateContent)


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger('Logger')
if os.getenv('X509_USER_CERT'):
    Rucio_cert = os.getenv('X509_USER_CERT')
    Rucio_key = os.getenv('X509_USER_KEY')
else:
    Rucio_cert = os.getenv('X509_USER_PROXY')
    Rucio_key = os.getenv('X509_USER_PROXY')
Rucio_host = 'https://cms-rucio.cern.ch'
Rucio_account = 'belforte'
Rucio_authUrl = 'https://cms-rucio-auth.cern.ch'
Rucio_caPath = '/etc/grid-security/certificates/'

nativeClient = Client(
    rucio_host=Rucio_host, auth_host=Rucio_authUrl, ca_cert=Rucio_caPath,
    account=Rucio_account,
    creds={"client_cert": Rucio_cert, "client_key": Rucio_key},
    auth_type = 'x509'
)
ret=nativeClient.ping()
print(ret['version'])
ret = nativeClient.whoami()
print(ret)

1.27.9
{'account_type': 'USER', 'suspended_at': None, 'created_at': '2020-04-28T23:25:20', 'account': 'belforte', 'status': 'ACTIVE', 'email': 'Stefano.Belforte@cern.ch', 'deleted_at': None, 'updated_at': '2020-04-28T23:25:20'}


In [3]:
username='belforte'
print(username)

belforte


In [4]:
# a shortcut
cli=nativeClient
# compatibility for CRAB calls
rucioClient = nativeClient

In [10]:
client = nativeClient
ALL_RSES = "ddm_quota>0&(tier=1|tier=2)&rse_type=DISK"
RSES_raw = client.list_rses(ALL_RSES)
print(type(RSES_raw))
rses=[]
for r in RSES_raw:
    rses.append(r['rse'])
print(type(rses))
len(rses)

<class 'generator'>
<class 'list'>


55

In [30]:
   
rses.sort()

for rse in rses:
    rse_attr = client.list_rse_attributes(rse)
    region = rse_attr["region"] if 'region' in rse_attr else None
    dq = int(rse_attr['ddm_quota'])//1e12
    use = 0
    tot = 0
    for ru in cli.get_rse_usage(rse):
        use = ru['total']//1e12 if ru['source'] == 'rucio' else use
        tot = ru['total']//1e12 if ru['source'] == 'static' else tot
            
    if dq > 10:
        print("%20s, %s, %s, %6d %6d %6d %6d" %
              (rse, rse_attr['country'], region, dq, use, tot, tot-use))

      T1_DE_KIT_Disk, DE, A,    640   6095   6750    655
      T1_ES_PIC_Disk, ES, C,    508   2577   3100    523
  T1_FR_CCIN2P3_Disk, FR, A,   1269   6722   8000   1278
     T1_IT_CNAF_Disk, IT, C,   1339   8055   9400   1345
      T1_UK_RAL_Disk, UK, A,    515   4724   5250    526
     T1_US_FNAL_Disk, US, B,   4737  24025  28750   4725
        T2_AT_Vienna, AT, A,     52    442    510     68
          T2_BE_IIHE, BE, C,    377   3532   3925    393
           T2_BE_UCL, BE, C,    160   1581   1757    176
        T2_BR_SPRACE, BR, B,    175   1809   2000    191
          T2_CH_CSCS, CH, A,    825   1592   2430    838
       T2_CN_Beijing, CN, D,    100    484    600    116
          T2_DE_DESY, DE, A,    336   5149   5500    351
          T2_DE_RWTH, DE, C,    186   1839   2040    201
       T2_EE_Estonia, EE, D,    124   1241   1380    139
        T2_ES_CIEMAT, ES, C,    190   2094   2300    206
          T2_ES_IFCA, ES, C,    183    652    850    198
           T2_FI_HIP, FI, D,   

In [15]:
print(rse_attr)

{'T2_US_Wisconsin': True, 'cms_type': 'real', 'country': 'US', 'ddm_quota': '336813721136324', 'fts': 'https://cmsfts3.fnal.gov:8446,https://fts3-cms.cern.ch:8446', 'lfn2pfn_algorithm': 'cmstfc', 'loadtest': True, 'pnn': 'T2_US_Wisconsin', 'quota_approvers': 'dasu,cvuosalo,mohapatr,dbradley', 'reaper': True, 'region': 'B', 'rule_approvers': 'dasu,cvuosalo,mohapatr,dbradley', 'source_for_total_space': 'static', 'source_for_used_space': 'rucio', 'tier': '2', 'update_from_json': True}


In [5]:
ret=cli.get_account_usage_history(username, 'T2_&')
print(ret)

RSENotFound: RSE does not exist.
Details: RSE 'T2_&' cannot be found in vo 'def'

In [ ]:
# in CRAB tape requests start with a block list
# start with a dataset on tape
dataset = '/MuonEG/Run2016F-v1/RAW'

# can't use DBS Api in this Jupyter. Ask Rucio
rucioResp = rucioClient.list_content('cms', dataset)
blocks = [item['name'] for item in rucioResp]

logger.info('found %d blocks', len(blocks))
pprint.pprint(blocks)


In [ ]:
dids = [{'scope':'cms','name':block} for block in blocks]
#dids = [{'scope':'user.belforte:','name':block} for block in blocks]
#pprint.pprint(dids)

In [ ]:
#print(len(blocks))
size = 0
for block in blocks:
    block_replicas = nativeClient.list_dataset_replicas('cms',block)
    bytes = block_replicas.next()['bytes']
    print (bytes)
    size += bytes
#replicas=list(block_replicas)
print('total = %d GBytes' % (size / 1e9))
print('total = %d TBytes' % (size / 1e12))

In [ ]:
#let's pick the small block
didsToRecall=[dids[7]]
bytesToRecall = cli.list_dataset_replicas('cms',blocks[7]).next()['bytes']
print(didsToRecall)
print ("%d MB" % (bytesToRecall/1.e6))

In [ ]:
# make a new container to hold them, embedding task name
taskName='210111_111111:belforte_crab_210111_111111'.replace(':','.')
print(taskName)
containerName='/TapeRecall/%s/USER' % taskName
print(containerName)
try:
    created = rucioClient.add_container('user.belforte', containerName)
except DataIdentifierAlreadyExists:
    logger.info("Container name already exists in Rucio. Keep going")
logger.info('container creation is %s', created)

In [ ]:
myScope = 'user.belforte'
containerDid = {'scope':myScope, 'name':containerName}

In [ ]:
# add blocks (datasets in Rucio language) to container
pprint.pprint(didsToRecall)
pprint.pprint(containerName)
#ret=nativeCli.add_datasets_to_container('user.belforte', containerName, didsToRecall)
#ret=nativeCli.add_containers_to_container('user.belforte', containerName, didsToRecall)
ret = rucioClient.attach_dids('user.belforte', containerName, didsToRecall)

In [ ]:
print(containerName)
cc=nativeClient.list_content('user.belforte',containerName)
content = [i for i in cc]
print(content)

In [ ]:
DAYS = 1 * 24 * 3600
# createReplicationRule(self, names, rseExpression, scope='cms', copies=1, **kwargs):
rules = nativeClient.add_replication_rule([containerDid], copies=1, rse_expression='T3_IT_Trieste',
                                  weight='ddm_quota', lifetime=DAYS, account=Rucio_account, grouping='ALL',
                                  activity='Analysis Input', comment='Staged from tape for %s' % username,
                                  ask_approval=True, asynchronous=True,
                                           )

In [ ]:

print(rules)
ruleId=rules[0]
print(ruleId)

In [ ]:
#ruleId='e7d17c2f768542318d414618f5d44f86'

In [ ]:
resp = nativeClient.get_replication_rule(ruleId,estimate_ttc=True)
print(resp)

In [ ]:
expiration=resp['expires_at']
print(expiration)

In [ ]:
pprint.pprint(resp)

In [ ]:
ruleInfo = rucioClient.get_replication_rule(ruleId,ttc=True)
state=ruleInfo['state']
"""
possible values:
class RuleState(Enum):
    REPLICATING = 'R'
    OK = 'O'
    STUCK = 'S'
    SUSPENDED = 'U'
    WAITING_APPROVAL = 'W'
    INJECT = 'I'
"""


In [ ]:
print(rules)
rule = rules[0]

print("Rule %s has been created for %s" % (rule, user))
print("This rule can be monitored through DAS by checking the dataset or directly through Rucio with:")
print(" rucio rule-info %s" % rule)

rule_info = cli.get_replication_rule(rule)

print('This rule expires at %s after which the data may be removed if not used occassionally'
      % rule_info['expires_at'])

print("This rule can be found by asking Rucio with:")
print(" rucio list-rules %s:%s" % ('user.belforte', containerName))

In [ ]:
if not pippo: print "SI"

In [ ]:
ruleId='cf9db5949a894efc9f39805a64ac275a'
resp=nativeClient.get_replication_rule(ruleId)

In [ ]:
rule_info=resp
rule = ruleId


print("Rule %s has been created for user %s" % (rule, user))
print("This rule can be monitored through DAS by checking the dataset or directly through Rucio with:")
print(" rucio rule-info %s" % rule)

rule_info = nativeClient.get_replication_rule(rule)

print('This rule expires at %s after which the data may be removed if not used'
      % rule_info['expires_at'])

print("This rule can be found by asking Rucio with:")
print(" rucio list-rules %s:%s" % ('user.belforte', containerName))

In [ ]:
pprint.pprint(resp)